In [7]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import os
import glob

In [8]:
folder_path = '../data/train_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [9]:
#file_list

In [10]:
participantid = []
currentgold = []
totalgold = []
xp = []
matchid = []
teamid = []


def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')


In [11]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

8000 files loaded


In [12]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 'currentGold': currentgold, 'totalGold':totalgold, 'xp': xp}
z_train = pd.DataFrame(di)
z_train

,matchId,teamId,participantId,currentGold,totalGold,xp
0,0,100,1,341,2741,4141
1,0,100,2,5,3045,2861
2,0,100,3,365,3545,4712
3,0,100,4,1214,2789,2894
4,0,100,5,566,2026,2551
...,...,...,...,...,...,...
79995,999,200,6,1127,1127,851
79996,999,200,7,844,1694,1546
79997,999,200,8,56,1731,2489
79998,999,200,9,212,1512,1257


In [13]:
z_train['matchId'].nunique()

8000

In [14]:
gold_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['currentGold'].mean()).reset_index()
gold_data

,matchId,teamId,currentGold
0,0,100,498.2
1,0,200,692.8
2,1,100,876.4
3,1,200,572.2
4,2,100,421.8
...,...,...,...
15995,7997,200,413.2
15996,7998,100,594.6
15997,7998,200,887.0
15998,7999,100,663.6


In [16]:
gold_data_piv = gold_data.pivot(index = 'matchId', columns = 'teamId', values = 'currentGold').reset_index()
gold_data_piv['gold_difference'] = gold_data_piv['100'] - gold_data_piv['200']
gold_data_piv
#rename(columns)

teamId,matchId,100,200,gold_difference
0,0,498.2,692.8,-194.6
1,1,876.4,572.2,304.2
2,2,421.8,492.2,-70.4
3,3,1058.4,863.0,195.4
4,4,856.2,458.0,398.2
...,...,...,...,...
7995,7995,610.8,360.6,250.2
7996,7996,641.6,596.0,45.6
7997,7997,607.0,413.2,193.8
7998,7998,594.6,887.0,-292.4


In [17]:
xp_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['xp'].mean()).reset_index()
#xp_data
xp_data_piv = xp_data.pivot(index = 'matchId', columns = 'teamId', values = 'xp').reset_index()
xp_data_piv['xp_diff'] = xp_data_piv['100'] - xp_data_piv['200']
xp_data_piv

teamId,matchId,100,200,xp_diff
0,0,3431.8,3754.6,-322.8
1,1,3554.0,3605.6,-51.6
2,2,3367.4,3493.4,-126.0
3,3,3512.4,3239.2,273.2
4,4,3652.8,3438.8,214.0
...,...,...,...,...
7995,7995,3760.6,3395.4,365.2
7996,7996,3459.6,3585.4,-125.8
7997,7997,3418.0,3144.2,273.8
7998,7998,3170.6,3730.8,-560.2


In [18]:
train_winners = pd.read_csv('../data/train_winners.csv')
train_winners.head()

,matchId,winner
0,0,200
1,1,200
2,2,200
3,3,100
4,4,200


In [19]:
gold_difs_wins = pd.merge(gold_data_piv, train_winners, on = 'matchId')

In [20]:
gold_xp= pd.merge(gold_difs_wins, xp_data_piv, on = 'matchId')

In [21]:
gold_xp

,matchId,100_x,200_x,gold_difference,winner,100_y,200_y,xp_diff
0,0,498.2,692.8,-194.6,200,3431.8,3754.6,-322.8
1,1,876.4,572.2,304.2,200,3554.0,3605.6,-51.6
2,2,421.8,492.2,-70.4,200,3367.4,3493.4,-126.0
3,3,1058.4,863.0,195.4,100,3512.4,3239.2,273.2
4,4,856.2,458.0,398.2,200,3652.8,3438.8,214.0
...,...,...,...,...,...,...,...,...
7995,7995,610.8,360.6,250.2,200,3760.6,3395.4,365.2
7996,7996,641.6,596.0,45.6,100,3459.6,3585.4,-125.8
7997,7997,607.0,413.2,193.8,200,3418.0,3144.2,273.8
7998,7998,594.6,887.0,-292.4,200,3170.6,3730.8,-560.2


In [22]:
X_feat = gold_xp[['gold_difference', 'xp_diff']]
y = gold_xp['winner']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_feat, y, 
                                                    test_size = 0.3,
                                                    random_state = 2,
                                                    stratify = y)

In [24]:
from sklearn.linear_model import LogisticRegression


In [25]:
logreg = LogisticRegression(random_state=2)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [26]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[860, 361],
       [454, 725]], dtype=int64)

In [46]:
from sklearn.metrics import classification_report
target_names = ['100', '200']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         100       0.65      0.70      0.68      1221
         200       0.67      0.61      0.64      1179

    accuracy                           0.66      2400
   macro avg       0.66      0.66      0.66      2400
weighted avg       0.66      0.66      0.66      2400



In [28]:
#test

In [29]:
folder_path = '../data/test_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [30]:
participantid = []
currentgold = []
totalgold = []
xp = []
matchid = []
teamid = []


def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')


In [31]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

2000 files loaded


In [32]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 'currentGold': currentgold, 'totalGold':totalgold, 'xp': xp}
z_test = pd.DataFrame(di)
z_test

,matchId,teamId,participantId,currentGold,totalGold,xp
0,8000,100,1,1513,3063,4659
1,8000,100,2,1742,4117,3539
2,8000,100,3,1015,4365,4431
3,8000,100,4,80,3555,3061
4,8000,100,5,78,2213,2409
...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621
19996,9999,200,7,1510,2860,2905
19997,9999,200,8,495,3245,4214
19998,9999,200,9,283,2483,2565


In [33]:
gold_data_test = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['currentGold'].mean()).reset_index()
gold_data_test

,matchId,teamId,currentGold
0,8000,100,885.6
1,8000,200,618.6
2,8001,100,610.4
3,8001,200,798.6
4,8002,100,634.2
...,...,...,...
3995,9997,200,830.8
3996,9998,100,812.6
3997,9998,200,631.8
3998,9999,100,708.2


In [34]:
gold_data_piv_tst = gold_data_test.pivot(index = 'matchId', columns = 'teamId', values = 'currentGold').reset_index()
gold_data_piv_tst['gold_difference'] = gold_data_piv_tst['100'] - gold_data_piv_tst['200']
gold_data_piv_tst
#rename(columns)

teamId,matchId,100,200,gold_difference
0,8000,885.6,618.6,267.0
1,8001,610.4,798.6,-188.2
2,8002,634.2,653.0,-18.8
3,8003,1003.8,654.2,349.6
4,8004,822.2,602.6,219.6
...,...,...,...,...
1995,9995,795.0,628.0,167.0
1996,9996,987.8,457.6,530.2
1997,9997,875.8,830.8,45.0
1998,9998,812.6,631.8,180.8


In [35]:
xp_data_test = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['xp'].mean()).reset_index()
#xp_data
xp_data_piv_test = xp_data_test.pivot(index = 'matchId', columns = 'teamId', values = 'xp').reset_index()
xp_data_piv_test['xp_diff'] = xp_data_piv_test['100'] - xp_data_piv_test['200']
xp_data_piv_test

teamId,matchId,100,200,xp_diff
0,8000,3619.8,3423.4,196.4
1,8001,3351.2,3305.4,45.8
2,8002,3328.0,3303.6,24.4
3,8003,3351.8,3434.6,-82.8
4,8004,3675.4,3422.6,252.8
...,...,...,...,...
1995,9995,3537.6,3569.0,-31.4
1996,9996,3654.6,3709.0,-54.4
1997,9997,3442.2,3777.6,-335.4
1998,9998,3659.6,3364.6,295.0


In [36]:
gold_xp_test= pd.merge(gold_data_piv_tst, xp_data_piv_test, on = 'matchId')

In [37]:
gold_xp_test

teamId,matchId,100_x,200_x,gold_difference,100_y,200_y,xp_diff
0,8000,885.6,618.6,267.0,3619.8,3423.4,196.4
1,8001,610.4,798.6,-188.2,3351.2,3305.4,45.8
2,8002,634.2,653.0,-18.8,3328.0,3303.6,24.4
3,8003,1003.8,654.2,349.6,3351.8,3434.6,-82.8
4,8004,822.2,602.6,219.6,3675.4,3422.6,252.8
...,...,...,...,...,...,...,...
1995,9995,795.0,628.0,167.0,3537.6,3569.0,-31.4
1996,9996,987.8,457.6,530.2,3654.6,3709.0,-54.4
1997,9997,875.8,830.8,45.0,3442.2,3777.6,-335.4
1998,9998,812.6,631.8,180.8,3659.6,3364.6,295.0


In [38]:
X_feat_test = gold_xp_test[['gold_difference', 'xp_diff']]

In [39]:
len(X_feat_test)

2000

In [40]:
y_pred_tst = logreg.predict(X_feat_test)

In [41]:
y_pred_tst

array([100, 100, 100, ..., 200, 100, 100], dtype=int64)

In [42]:
len(y_pred_tst)

2000

In [43]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,100
...,...,...
1995,9995,100
1996,9996,100
1997,9997,100
1998,9998,100


In [44]:
submission['winner'] = y_pred_tst
submission

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,200
4,8004,100
...,...,...
1995,9995,200
1996,9996,200
1997,9997,200
1998,9998,100


In [45]:
submission.to_csv('../data/submissionV3.csv', index = False)